# Derive the Dynamics and Sensor Model

## Derive Dynamics Model

Import all packages used to derive the equations of motion of the system

In [3]:
import sympy as sym
import numpy as np
sym.init_printing()

First we need to find the value of the acceleration due to gravity on the planet on which the quadrotor is operating. We can do this using a function provided by the ae353_quadrotor module. You must select from one of the following planets: "Mercury", "Venus", "Earth", "Moon", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto".

In [4]:
# Get the gravity in m/s^2
from ae353_quadrotor import get_gravity
planet = "Earth"
grav = get_gravity(planet, verbose=True)

Define all the system constants (system parameters)

In [5]:
# Mass and moment of inertia of the bus(all need to be properly dervided from actual things)
#if you want to test values you can use these numbers and comment out they sympy line
"""mass = 0.5    # The mass of the quadrotor in kg
Ixx = 0.0023  # The moment of inertia of the quadrotor in 
Iyy = 0.0023  # The mass of the quadrotor in kg
Izz = 0.0040  # The mass of the quadrotor in kg
lxy = 0.25    # The x or y distance from the quadrotor frame to the mocap markers in meters
lz = 0.046875 # The z distance from the quadrotor frame to the mocap markers in meters
k = 1.2       # The lift constant
b = 0.8       # The drag coefficient"""
mass,Ixx,Iyy,Izz,lxy,lz,k,b = sym.symbols("mass,Ixx,Iyy,Izz,l,lz,k,b")

Generate the symbols that define the state of the quadrotor. Also, create the rotation matrices that define the orientation of the quadrotor in the world coordinate system.

In [ ]:
# Define position
px_inW, py_inW, pz_inW = sym.symbols('p_x, p_y, p_z')
p_inW = sym.Matrix([[px_inW],
                    [py_inW],
                    [pz_inW]])

# Define velocity
vx_inB, vy_inB, vz_inB = sym.symbols('v_x, v_y, v_z')
v_inB = sym.Matrix([[vx_inB],
                    [vy_inB],
                    [vz_inB]])

# Define roll (phi), pitch (theta), and yaw (psi) angles
phi, theta, psi = sym.symbols('phi, theta, psi')
n = sym.Matrix([[phi],
                [theta],
                [psi]])


# Define angular velocities
wx_inB, wy_inB, wz_inB = sym.symbols('omega_x, omega_y, omega_z') # same as the angle dots
w_inB = sym.Matrix([[wx_inB],
                    [wy_inB],
                    [wz_inB]])

# Define roll, pitch, and yaw rotation matrices that describe the bus's orientation in the 
# world frame
Rx = sym.Matrix([[1,            0,             0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi),  sym.cos(phi)]])
Ry = sym.Matrix([[ sym.cos(theta), 0, sym.sin(theta)],
                 [              0, 1,              0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi),  sym.cos(psi), 0],
                 [           0,             0, 1]])
R_ofB_inW = Rz @ Ry @ Rx
R_ofW_inB = R_ofB_inW.T
R_ofB_inW

Calculate the roll, pitch, and yaw rates as a function of the orientation and body-fixed angular rates.

In [ ]:
# Define the transformation that takes roll, pitch, and yaw rates to body-fixed angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.Matrix.hstack(ex, Rx.T@ey, (Ry@Rx).T@ez)

# Invert the transformation to get a matrix that takes body-fixed angular rates to 
# roll, pitch, and yaw rates
M_inv = sym.simplify(M.inv())

# Convert the body-fixed angular rates to roll, pitch, and yaw rates
rpy_dot = sym.simplify(M_inv@w_inB) #ndot as described by the paper (4)
print("Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:")
sym.N(rpy_dot,3)

Next we assemble the moment of inertia tensor.

In [ ]:
# Assemble the moment of inertia matrix
I_inB = sym.Matrix([[Ixx, 0.,  0.],
                    [0.,  Iyy, 0.],
                    [0.,  0.,  Izz]])

Generate the symbols that define the input to the system. Also sum the together to get the net torque vector and force vector.

In [ ]:
# Define inputs
taux_inB, tauy_inB, tauz_inB, fz_inB,w1,w2,w3,w4 = sym.symbols('tau_x, tau_y, tau_z, f_z,w1,w2,w3,w4')
#Fz is the same as total thrust T in paper(eq7)
fz_inB = k* (w1**2+w2**2+w3**2+w4**2)
#these taus are defined by eq (8) in the paper
taux_inB = lxy*k*(-w2**2+w4**2)
tauy_inB = lxy*k*(-w1**2+w3**2)
tauz_inB = b*(w1**2+w2**2+w3**2+w4**2) 


#w_i_dot is negligible in eq (6) so the other part of the term can be disregarded 
# Get the net torque vector
tau_inB = sym.Matrix([[taux_inB],
                      [tauy_inB],
                      [tauz_inB]])
tau_inB = sym.simplify(tau_inB)
#in the paper this is equated to tau_B, the angular velocities of each rotor and the total torque T_Mi

# Get the gravity force and convert to body coordiantes
grav_inW = sym.Matrix([[0.],
                       [0.],
                       [-mass * grav]])
grav_inB = R_ofW_inB @ grav_inW

# Get the net force vector (same as component G + total thrust as described by paper in like eq (9))
f_inB = grav_inB + sym.Matrix([[0.],
                               [0.],
                               [fz_inB]])
f_inB = sym.simplify(f_inB)
tau_inB


Convert the body-fixed velocity to world-fixed velocity to get the derivatives of position.

In [ ]:
v_inW = R_ofB_inW @ v_inB
xyz_dot = sym.simplify(v_inW)
print("px, py, pz rate as a function of orientation and body-fixed velocities:")
sym.N(xyz_dot,3)

Apply [Newton's second law in a rotating frame](https://en.wikipedia.org/wiki/Rotating_reference_frame) to get the derivative of the velocity in body-fixed coordinates.

In [ ]:
#**BODY FRAME**
v_inB_dot = (1 / mass) * (f_inB - w_inB.cross(mass * v_inB)) #this solves eq (9)
v_inB_dot = sym.simplify(v_inB_dot)
print("Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:")
sym.N(v_inB_dot,3) #same as vdot in the paper eq(11)

In [ ]:
x_ddot = (1/mass)*(R_ofB_inW@f_inB)
x_ddot = sym.simplify(x_ddot)
x_ddot

Use [Euler's Equations](https://en.wikipedia.org/wiki/Euler%27s_equations_(rigid_body_dynamics)) to generate the equations of motion that relate angular velocity to input torque.

In [ ]:
# Apply Euler's equation to get the time derivative of the angular velocities of the
# quadrotor in the quadrotor frame **BODY FRAME**
w_inB_dot = I_inB.inv() @ (tau_inB - w_inB.cross(I_inB@w_inB)) #eq (11 in paper), but we ignore gyroscopic forces
w_inB_dot = sym.simplify(w_inB_dot)
print("Time derivative of the angular velocities in the body-fixed frame:")
sym.N(w_inB_dot, 3)

In [ ]:
#**INERTIAL FRAME**
M_inv_dot = np.array([[0,rpy_dot[0]*sym.cos(phi)*sym.tan(theta) + rpy_dot[1] * sym.sin(phi)/(sym.cos(theta)**2), -rpy_dot[0] * sym.sin(phi) *sym.cos(theta)  + rpy_dot[1]*sym.cos(phi)/(sym.cos(theta)**2)],
                    [0, -rpy_dot[0] * sym.sin(phi), -rpy_dot[0] * sym.cos(phi)],
                    [0, rpy_dot[0]*sym.cos(phi)/sym.cos(theta) + rpy_dot[0]*sym.sin(phi)*sym.tan(theta)/sym.cos(theta), -rpy_dot[0]*sym.sin(phi)/sym.cos(theta) + rpy_dot[1]*sym.cos(phi)*sym.tan(theta)/sym.cos(theta)]])
rpy_ddot = M_inv_dot@w_inB + M_inv@w_inB_dot
rpy_ddot = sym.simplify(rpy_ddot) #eq (12)
print("Time derivative of roll, pitch, yaw rates in the body-fixed frame")
sym.N(rpy_ddot,3)

# Euler-Lagrange eqs
These solve for the equations in the inertial frame so x_ddot2 and rpy_ddot2 should be the same as x_ddot and rpy_ddot (idk if we want in inertial or body but if body, these can be neglected)

In [ ]:
#jacobian matrix
J = M.T @ I_inB @ M
J = sym.simplify(J)
J

J_inv = sym.simplify(J.inv())

In [ ]:
E_trans = (mass/2)*xyz_dot.T@xyz_dot
E_rot = (1/2)*w_inB.T@I_inB@w_inB
E_pot = mass * grav * sym.Matrix([p_inW[2]])
L = E_trans + E_rot - E_pot
sym.N(L,3)

In [ ]:
x_ddot2 = (1/mass)*(R_ofB_inW@f_inB)
x_ddot2 = sym.simplify(x_ddot2)
x_ddot2

In [ ]:
#coriolis term (not symmetric)
C11 = 0
C12 = (Iyy-Izz)*(rpy_dot[1]*sym.cos(phi)*sym.sin(phi)+rpy_dot[2]*sym.cos(theta)*sym.sin(phi)**2)+(Izz-Iyy)*rpy_dot[2]*sym.cos(theta)*sym.cos(phi)**2-Ixx*rpy_dot[2]*sym.cos(theta)
C13 = (Izz-Iyy)*rpy_dot[2]*sym.cos(phi)*sym.sin(phi)*sym.cos(theta)**2
C21 = (Izz-Iyy)*(rpy_dot[1]*sym.cos(phi)*sym.sin(phi)+rpy_dot[2]*sym.cos(theta)*sym.sin(phi))+(Iyy-Izz)*rpy_dot[2]*sym.cos(theta)*sym.cos(phi)**2+Ixx*rpy_dot[2]*sym.cos(theta)
C22 = (Izz-Iyy)*rpy_dot[0]*sym.cos(phi)*sym.sin(phi)
C23 = -Ixx*rpy_dot[2]*sym.sin(theta)*sym.cos(theta) + Iyy*rpy_dot[2]*sym.sin(theta)*sym.cos(theta)*sym.sin(phi)**2 + Izz*rpy_dot[2]*sym.sin(theta)*sym.cos(theta)*sym.cos(phi)**2
C31 = C13 - Ixx*rpy_dot[1]*sym.cos(theta)
C32 = (Izz-Iyy)*(rpy_dot[1]*sym.cos(phi)*sym.sin(phi)*sym.sin(theta)+rpy_dot[0]*sym.cos(theta)*sym.sin(phi)**2)+(Iyy-Izz)*(rpy_dot[0]*sym.cos(theta)*sym.cos(phi)**2)-1*C23
C33 = (Iyy-Izz)*rpy_dot[0]*sym.cos(phi)*sym.sin(phi)*sym.cos(theta)**2 - rpy_dot[1]*sym.sin(theta)*sym.cos(theta)*(Iyy*sym.sin(phi)**2 + Izz*sym.cos(phi)**2 - Ixx)
C = sym.Matrix([[C11,C12,C13],
                [C21,C22,C23],
                [C31,C32,C33]])
C

In [ ]:
rpy_ddot2 = J_inv@(tau_inB - C@rpy_dot)
rpy_ddot2 = sym.simplify(rpy_ddot2)
sym.N(rpy_ddot2,3)

# F matrix
f is all of the eoms in the body frame and consists of linear velocities, linear accelerations, angular velocities, angular accelerations in body frame

Assemble all differential equations.

In [ ]:
f = sym.Matrix.vstack(xyz_dot,
                      v_inB_dot,
                      rpy_dot,
                      w_inB_dot)

The equations of motion have this form:

$$\begin{bmatrix} \dot{p_x} \\ \dot{p_y} \\ \dot{p_z} \\ \dot{v_x} \\ \dot{v_y} \\ \dot{v_z} \\ \dot{\phi} \\ \dot{\theta} \\ \dot{\psi} \\ \dot{\omega_x} \\ \dot{\omega_y} \\ \dot{\omega_z} \end{bmatrix}=f\left(p_x, p_y, p_z, v_x, v_y, v_z, \phi, \theta, \psi, \omega_x, \omega_y, \omega_z, \tau_x, \tau_y, \tau_z, f_z\right)$$

Here is the function $f$:

In [ ]:
sym.N(f,3)